# Setup environment

In [1]:
# Essentials
import os
import numpy as np
import importlib
import json
from datetime import datetime

# PyTorch
import torch
from torch.utils.data import DataLoader

# Utils
import utils
importlib.reload(utils)

from utils import get_dict, get_files, check_corrupted_imgs, perturb_image, save_image, make_dirs, get_model, select_gpu, get_data
from utils import CustomTransforms, My_data, FocalLoss

# OnePixelAttack
import OnePixelAttack
importlib.reload(OnePixelAttack)

# TriangleAttack
import TriangleAttack
importlib.reload(TriangleAttack)

<module 'TriangleAttack' from '/home/jovyan/TriangleAttack.py'>

## Setup Cuda

In [2]:
# Set a higher max split size to avoid memory problems
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [3]:
device = torch.device(f"cuda:{select_gpu()}" if torch.cuda.is_available() else "cpu")

print(device)

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(torch.cuda.memory_summary(device=None, abbreviated=False))
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

Selecting GPU 0 with 13052MB free memory
cuda:0
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from lar

## Get the data

In [4]:
# Load train and test files that are used for the model.
train_dict, test_dict = get_data(os.path.join(os.getcwd(), "BreaKHis_v1"))

Opening /home/jovyan/txt/train.txt.txt
Opening /home/jovyan/txt/test.txt.txt


In [5]:
test_files = "./dataset/test/**/**/*.png"

## Setup Transformers

In [6]:
custom_transforms = CustomTransforms()
resize_transform = custom_transforms.get_transform("resize_tensor")
test_transform = custom_transforms.get_transform("test")

## Test performance of Network

In [7]:
filepaths = {
    "Network": ["SWIN", "RESNET"],
    "Attack": ["Regular", "OnePixel", "Triangle"]
}

In [8]:
org_norm = My_data(get_files(test_files), transforms=test_transform)
org_dataloader = DataLoader(org_norm)

plot_data = {
    str(i+1): [] for i in range(org_norm.__len__())
}

In [9]:
model_name = "swin"
model = get_model(device, test_dict, model_name)
model.eval()

/home/jovyan/.local/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SwinTransformerV2(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerV2Stage(
      (downsample): Identity()
      (blocks): ModuleList(
        (0): SwinTransformerV2Block(
          (attn): WindowAttention(
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=3, bias=False)
            )
            (qkv): Linear(in_features=96, out_features=288, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (drop_path1): Identi

In [22]:
with torch.no_grad():
    for i, (image, label) in enumerate(org_dataloader):
        path = org_norm.__getpath__(i)
        true_label = torch.argmax(label.to(device), dim=1)
        confs = model(image.to(device))
        pred_label = torch.argmax(confs, dim=1)
    
        plot_data[str(i)] = [path, true_label.item(), true_label.item(), confs.cpu().numpy()[0].tolist()]

In [23]:
name = "{} - {} - {}".format(filepaths["Network"][0], filepaths["Attack"][0], datetime.now().strftime("%Y-%m-%d %H%M"))

with open("{}.txt".format(name), "w") as output:
    output.write(json.dumps(plot_data))

# Advanced OnePixel Attack

In [24]:
op_advs = []
with torch.no_grad():
    for i, (image, label) in enumerate(org_dataloader):
        op_adv = OnePixelAttack.attack(i, model, device, image, label, pixel_count=1, maxiter=50, popsize=15)
        op_advs.append(op_adv)


 Result Maximum number of iterations has been exceeded.
Success False
tensor([[  3.6850,  -5.0853,  -8.4693,  -3.9080,  -8.6841,  -9.1027, -11.4568,
         -11.3317]]) tensor([[  2.2942,  -5.2510,  -6.3856,  -2.8770,  -7.3100,  -8.5122, -10.8061,
         -10.3221]])
0 0

 Result Maximum number of iterations has been exceeded.
Success False
tensor([[  4.5329,  -6.1741,  -8.2312,  -5.2091,  -6.6450,  -9.8714, -10.2602,
         -12.0044]]) tensor([[  3.7683,  -6.0481,  -8.1165,  -4.1261,  -6.6781, -10.4078, -10.1751,
         -11.9796]])
0 0


KeyboardInterrupt: 

In [ ]:
plot_data = {
    str(i+1): [] for i in range(org_norm.__len__())
}

In [ ]:
make_dirs("one_pixel_attack")
with torch.no_grad():
    for i, (image, label) in enumerate(org_dataloader):
        path = org_norm.__getpath__(i)
        true_label = torch.argmax(label.to(device), dim=1)
        confs = model(op_advs[i].to(device))
        pred_label = torch.argmax(confs, dim=1)
        save_image(op_advs[i], "./one_pixel_attack/" + path)
        plot_data[str(i)] = [path, true_label.item(), true_label.item(), confs.cpu().numpy()[0].tolist()]

In [ ]:
name = "{} - {} - {}".format(filepaths["Network"][0], filepaths["Attack"][1], datetime.now().strftime("%Y-%m-%d %H%M"))

with open("{}.txt".format(name), "w") as output:
    output.write(json.dumps(plot_data))

# Triangle attack   

In [ ]:
import os
os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = "/usr/bin/git"

import git
git.refresh("/usr/bin/git")

from foolbox import PyTorchModel
from TriangleAttack import TA


org_norm = My_data(get_files(test_files), transforms=test_transform)
org_dataloader = DataLoader(org_norm)


In [ ]:
model_name = "swin"
model = PyTorchModel(get_model(device, test_dict, model_name).eval(), bounds=(0,1), device=device)
# run on a10 gpu as it has highest memory
print("Attack !")

with torch.no_grad():
    ta_model = TA(model, input_device=device)
    my_advs, q_list, my_intermediates, max_length = ta_model.attack(org_dataloader)
    print('TA Attack Done')

In [ ]:
plot_data = {
    str(i+1): [] for i in range(len(my_advs))
}

make_dirs("triangle_attack")
for i in len(my_advs):
    path = org_norm.__getpath__(i)
    true_label = torch.argmax(org_norm.__getitem__(i)[1], dim=1)
    confs = model(my_advs[i])
    pred_label = torch.argmax(confs, dim=1)
    
    save_image(my_advs, "./triangle_attack/" + path)
    plot_data[str(i)] = [path, true_label, pred_label, confs]

In [ ]:
name = "{} - {} - {}".format(filepaths["Network"][0], filepaths["Attack"][2], datetime.now().strftime("%Y-%m-%d %H%M"))

with open("{}.txt".format(name), "w") as output:
    output.write(json.dumps(plot_data))

# Test Area

In [ ]:
# import albumentations
# import albumentations.pytorch

# transforms_a =  albumentations.Compose([
#                 albumentations.Resize(256, 256),
#                 albumentations.Normalize(mean=[0, 0, 0], std=[255, 255, 255], max_pixel_value=1.0),
#                 albumentations.pytorch.transforms.ToTensorV2()
#             ])

# test_a= My_data([test_files[0]], transforms_a)

# save_image(test_a.__getitem__(0)[0], "attemptofsucces_a.png")

# transforms_b =  albumentations.Compose([
#                 albumentations.Resize(256, 256),
#                 albumentations.Normalize(mean=[0 , 0, 0], std=[1, 1, 1], max_pixel_value=255),
#                 albumentations.pytorch.transforms.ToTensorV2()
#             ])

# test_b = My_data([test_files[0]], transforms_b)

# save_image(test_b.__getitem__(0)[0], "attemptofsucces_b.png")

In [ ]:
# print(org_dataloader.dataset)

In [ ]:
i = 0
for img in my:
    save_image(img, "dataset/ta_attack/" + str(i) + "ta.png")
    i += 1
    

In [ ]:
save_image(org_norm.__getitem__(0)[0], "0org.png")

In [ ]:
print(my_advs[753], q_list[753], my_intermediates[753], max_length)

In [ ]:
torch.save(my_advs, "TA_advs.pt")

In [ ]:
torch.save(q_list, "q_list.pt")

In [ ]:
torch.save(my_intermediates, "my_intermediates.pt")

In [ ]:
torch.save(max_length, "max_length.pt")

In [ ]:
data = My_data(["0org.png", "0ta.png", "753org.png", "753ta.png"])

In [ ]:
import cv2

img_org = cv2.imread("0org.png")
img_ta = cv2.imread("2700ta.png")

In [ ]:
mask = img_org != img_ta

In [ ]:
mask.sum()

In [ ]:
my_advs = torch.load("TA_advs.pt", device)

In [ ]:
my_advs[0]

In [ ]:
save_image(org_norm.__getitem__(270)[0], "270org.png")